In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os
from PIL import Image

In [4]:
Training_original="/content/drive/MyDrive/DEEPFAKE AUDIO/k47yd3m28w-4/Training_original/Training_original/Training_original"
Training_Fake="/content/drive/MyDrive/DEEPFAKE AUDIO/k47yd3m28w-4/Training_original/Training_original/Training_original"
Validation_original="/content/drive/MyDrive/DEEPFAKE AUDIO/k47yd3m28w-4/Validation_original/Validation_original/Validation_original"
Validation_Fake="/content/drive/MyDrive/DEEPFAKE AUDIO/k47yd3m28w-4/Validation_fake/Validation_fake/Validation_fake"

In [5]:
images_original = []
images_fake = []

In [6]:


# Loop through all files in the folder
for filename in os.listdir(Training_original):
    if filename.endswith((".jpg")):  # Check for common image extensions
        # Open the image file
        image_path = os.path.join(Training_original, filename)
        img = Image.open(image_path)
        images_original.append(img)

# Output the number of images loaded
print(f"Total number of Training original images loaded: {len(images_original)}")


# Loop through all files in the folder
for filename in os.listdir(Training_Fake):
    if filename.endswith((".jpg")):  # Check for common image extensions
        # Open the image file
        image_path = os.path.join(Training_Fake, filename)
        img = Image.open(image_path)
        images_fake.append(img)

# Output the number of images loaded
print(f"Total number of Training fake images loaded: {len(images_fake)}")

Total number of Training original images loaded: 2036
Total number of Training fake images loaded: 2036


In [7]:
import numpy as np
from sklearn.utils import shuffle

# Assuming images are PIL Image objects, resize them if needed to reduce memory usage
def preprocess_image(image, target_size=(128, 128)):
    # Resize and convert to numpy array
    return np.array(image.resize(target_size)) / 255.0  # Normalize pixel values

# Preprocess images to reduce memory usage
images_original = [preprocess_image(img) for img in images_original]
images_fake = [preprocess_image(img) for img in images_fake]

# Label the data
real_labels = [0] * len(images_original)  # Label 0 for original images
fake_labels = [1] * len(images_fake)  # Label 1 for fake images

# Convert the lists directly to numpy arrays for more efficient memory management
images = np.array(images_original + images_fake, dtype=np.float32)
labels = np.array(real_labels + fake_labels, dtype=np.int32)

# Shuffle the combined dataset
images, labels = shuffle(images, labels, random_state=42)



In [8]:
val_original=[]
val_fake=[]

In [9]:
# Loop through all files in the folder
for filename in os.listdir(Validation_original):
    if filename.endswith((".jpg")):  # Check for common image extensions
        # Open the image file
        image_path = os.path.join(Validation_original, filename)
        img = Image.open(image_path)
        val_original.append(img)

# Output the number of images loaded
print(f"Total number of Validation original images loaded: {len(val_original)}")


# Loop through all files in the folder
for filename in os.listdir(Validation_Fake):
    if filename.endswith((".jpg")):  # Check for common image extensions
        # Open the image file
        image_path = os.path.join(Validation_Fake, filename)
        img = Image.open(image_path)
        val_fake.append(img)

# Output the number of images loaded
print(f"Total number of Validation fake images loaded: {len(val_fake)}")

Total number of Validation original images loaded: 870
Total number of Validation fake images loaded: 864


In [10]:

# Preprocess images to reduce memory usage
val_original = [preprocess_image(img) for img in val_original]
val_fake = [preprocess_image(img) for img in val_fake]

# Label the data
Val_real_labels = [0] * len(val_original)  # Label 0 for original images
Val_fake_labels = [1] * len(val_fake)  # Label 1 for fake images

# Convert the lists directly to numpy arrays for more efficient memory management
Val_images = np.array(val_original + val_fake, dtype=np.float32)
Val_labels = np.array(Val_real_labels + Val_fake_labels, dtype=np.int32)

# Shuffle the combined dataset
Val_images, Val_labels = shuffle(Val_images, Val_labels, random_state=42)



In [11]:
len(real_labels)

2036

In [12]:
len(images)

4072

In [13]:
images[0].size

49152

In [14]:
Val_images[0].size

49152

In [15]:
Val_images.shape

(1734, 128, 128, 3)

In [16]:
images.shape

(4072, 128, 128, 3)

In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Function to create a residual block
def residual_block(x, filters, shortcut_filters=None): # Add shortcut_filters argument
    shortcut = x  # Save the input for the skip connection

    # If shortcut_filters is provided, apply 1x1 convolution to match dimensions
    if shortcut_filters:
        shortcut = layers.Conv2D(filters, (1, 1), padding='same')(shortcut)

    x = layers.Conv2D(filters, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(filters, (3, 3), padding='same')(x)
    x = layers.add([x, shortcut])  # Add the input back to the output
    x = layers.Activation('relu')(x)
    return x

# ResNet model definition
def resnet_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # Initial convolution
    x = layers.Conv2D(64, (7, 7), padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D((3, 3))(x)

    # Residual blocks
    filters_list = [64, 128, 256]
    for i, filters in enumerate(filters_list):
        for _ in range(2):  # Two residual blocks for each filter size
            # Pass shortcut_filters only when filters change
            shortcut_filters = filters_list[i-1] if i > 0 else None
            x = residual_block(x, filters, shortcut_filters)
        x = layers.MaxPooling2D((2, 2))(x)

    # Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)

    # Fully connected layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs=inputs, outputs=outputs)

In [18]:
# Instantiate the model with the correct input shape
input_shape = (128, 128, 3)  # Example input shape
num_classes = 2  # Binary classification
resnet = resnet_model(input_shape, num_classes)

# Compile the model
resnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
resnet.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 128, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 128, 128, 64)   │          9,472 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 42, 42, 64)     │              0 │ conv2d[0][0]           │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 42, 42, 64)     │         36,928 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 42, 42, 64)     │         36,928 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 42, 42, 64)     │              0 │ conv2d_2[0][0],        │
│                           │                        │                │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 42, 42, 64)     │              0 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 42, 42, 64)     │         36,928 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 42, 42, 64)     │         36,928 │ conv2d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 42, 42, 64)     │              0 │ conv2d_4[0][0],        │
│                           │                        │                │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 42, 42, 64)     │              0 │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 21, 21, 64)     │              0 │ activation_1[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 21, 21, 128)    │         73,856 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 21, 21, 128)    │        147,584 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 21, 21, 128)    │          8,320 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 21, 21, 128)    │              0 │ conv2d_7[0][0],        │
│                           │                        │                │ conv2d_5[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 21, 21, 128)    │              0 │ add_2[0][0]            │
├──────────────────────

 Total params: 2,863,362 (10.92 MB)

 Trainable params: 2,863,362 (10.92 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Compile the model
resnet = resnet_model(input_shape=(128, 128, 3), num_classes=2)  # Define input_shape according to your data
resnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [20]:
labels = tf.keras.utils.to_categorical(labels, num_classes=2)
Val_labels = tf.keras.utils.to_categorical(Val_labels, num_classes=2)

In [21]:
# Train the model
history = resnet.fit(
    images, labels,
    validation_data=(Val_images, Val_labels),
    epochs=30,
    batch_size=32
)


Epoch 1/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 617s 5s/step - accuracy: 0.5113 - loss: 0.8872 - val_accuracy: 0.4983 - val_loss: 0.6932
Epoch 2/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 635s 5s/step - accuracy: 0.4755 - loss: 0.6932 - val_accuracy: 0.4983 - val_loss: 0.6932
Epoch 3/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 666s 5s/step - accuracy: 0.5037 - loss: 0.6932 - val_accuracy: 0.4983 - val_loss: 0.6932
Epoch 4/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 636s 5s/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.4983 - val_loss: 0.6932
Epoch 5/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 620s 5s/step - accuracy: 0.4918 - loss: 0.6932 - val_accuracy: 0.5017 - val_loss: 0.6931
Epoch 6/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 623s 5s/step - accuracy: 0.4942 - loss: 0.6932 - val_accuracy: 0.4983 - val_loss: 0.6931
Epoch 7/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 626s 5s/step - accuracy: 0.5041 - loss: 0.6932 - val_accuracy: 0.4983 - val_loss: 0.6932
Epoch 8/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 614s 5s/step - accuracy: 0.4894 - loss: 0.6932 - val_accu

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# Plotting training & validation accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plotting training & validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()
